In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths
import os
import sys
import numpy as np
import argparse
import pprint
import pdb
import time
import logging

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from datetime import datetime

import torchvision.transforms as transforms
from torch.utils.data.sampler import Sampler

from roi_data_layer.roidb import combined_roidb
from roi_data_layer.roibatchLoader import roibatchLoader
from model.utils.config import cfg, cfg_from_file, cfg_from_list, get_output_dir
from model.utils.net_utils import weights_normal_init, save_net, load_net, \
      adjust_learning_rate, save_checkpoint, clip_gradient

from model.fpn.resnet import resnet
import pdb

TypeError: object of type 'int' has no len()

In [10]:
imdb, roidb, ratio_list, ratio_index = combined_roidb('voc_2007_trainval')
FPN = resnet(imdb.classes, 101, pretrained=True, class_agnostic=True)

Loaded dataset `voc_2007_trainval` for training
Set proposal method: gt
Appending horizontally-flipped training examples...
voc_2007_trainval gt roidb loaded from /workspace/mnt/group/customization/pengyuyan/IoU-Net/fpn.pytorch/data/cache/voc_2007_trainval_gt_roidb.pkl
done
Preparing training data...
done
before filtering, there are 10022 images...
after filtering, there are 10022 images...


In [12]:
FPN.create_architecture()

Loading pretrained weights from data/pretrained_model/resnet101_caffe.pth


In [19]:
FPN.forward

<bound method _FPN.forward of resnet(
  (maxpool2d): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
  (RCNN_rpn): _RPN_FPN(
    (RPN_Conv): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (RPN_cls_score): Conv2d(512, 6, kernel_size=(1, 1), stride=(1, 1))
    (RPN_bbox_pred): Conv2d(512, 12, kernel_size=(1, 1), stride=(1, 1))
    (RPN_proposal): _ProposalLayer_FPN()
    (RPN_anchor_target): _AnchorTargetLayer_FPN()
  )
  (RCNN_proposal_target): _ProposalTargetLayer()
  (RCNN_roi_pool): _RoIPooling()
  (RCNN_roi_align): RoIAlignAvg()
  (RCNN_roi_prroi): PrRoIPool2D()
  (RCNN_roi_crop): _RoICrop()
  (RCNN_layer0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  )
  (RCNN_layer1): Sequential(
   

In [15]:
FPN.RCNN_cls_score

Linear(in_features=1024, out_features=21, bias=True)

In [16]:
FPN.RCNN_bbox_pred

Linear(in_features=1024, out_features=4, bias=True)

In [17]:
FPN.RCNN_top

Sequential(
  (0): Conv2d(256, 1024, kernel_size=(7, 7), stride=(7, 7))
  (1): ReLU(inplace)
  (2): Conv2d(1024, 1024, kernel_size=(1, 1), stride=(1, 1))
  (3): ReLU(inplace)
)